<a href="https://colab.research.google.com/github/rpasquini/metodos_cuantitativos/blob/main/notebooks/R/5_Modelos_de_Clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
%load_ext rpy2.ipython


In [ ]:
%%R
# Install required packages if not already installed
if (!require("nnet")) install.packages("nnet")  # For multinomial logistic regression
if (!require("sf")) install.packages("sf")      # For spatial data
if (!require("dplyr")) install.packages("dplyr") # For data manipulation
if (!require("ggplot2")) install.packages("ggplot2") # For plotting
if (!require("margins")) install.packages("margins") # For marginal effects

# Load the packages
library(nnet)
library(sf)
library(dplyr)
library(ggplot2)
library(margins)


## Estimación de un Modelo de Elección de "Ubicación Residencial" adaptado a una situación de vivir en la formalidad vs. la informalidad.

A modo de ejemplo, vamos a intentar explicar predecir la localización de los hogares en las áreas de la "ciudad formal" o en los "asentamientos informales". Vamos a utilizar un extracto de los datos de la investigación de Goytia, Heikkila, Pasquini (2020).


## Descripción de variables
* bformal es una dummy que señala con 1 a las areas "formales" y 0 a las areas "informales". Este es un criterio de delimitación espacial.  "Informal" se utiliza para el área que constituye un barrio popular  
* dconurb1, dconurb2, dconurb3 son dummies que señalan a los agrupamientos de partidos del AMBA "Conurbano 1", "Conurbano 2" y "Conurbano 3" respectivamente, según la categorización del INDEC.  Vease https://www.indec.gob.ar/dbindec/folleto_gba.pdf
* dprop_soloviv', 'dprop_ocupant', 'dinquilinos' son dummies que señalan a los que son propietarios solo de la vivienda, ocupantes e inquilinos respectivamente.
* dedu_secincomp', 'dedu_seccomp', 'dedu_ter_o_mas' son dummies que señalan la educación del jefe de hogar
* 'dconexionagua1', 'delectricidad_med', 'dconexiongas'  son dummies que señalan la presencia de conexión al agua de red, la presencia de un medidor de electricidad y la conexión al gas de red respectivamente.


El modelo que propondremos consistirá en explicar la condición de vivir en el "área formal" a partir de un conjunto de atributos del hogar, incluyendo el ingreso, el tamaño, y un conjunto de variabes indicadores del nivel educativo del jefe de hogar.

Proponemos un Modelo Logit. Recordemos que este modelo puede pensar como el modelo de regresión lineal que es transformado mediante la función logística:

$$P(F=1)_i=\frac{1}{1+e^{-Z_i}} $$

donde

$$Z_i=\beta_0+\beta_1IngresoHogar_i+\beta_2TamañoHogar_i+\beta_3DSecundarioCompleto_i+\beta_4DTerciarioCompleto_i+\epsilon_i$$


**Observación:** Una propiedad de este modelo que puede ser útil más adelante es notar que este modelo cumple que el *ratio de las chances* de vivir en el sector formal (definido en este caso como la probabilidad de F=1 sobre la probabilidad de F=0) tomado en logaritmo se asume de forma lineal:


$$\log(\frac{P(F=1)}{P(F=0)})=\beta_0+\beta_1IngresoHogar_i+\beta_2TamañoHogar_i+\beta_3DSecundarioCompleto_i+\beta_4DTerciarioCompleto_i+\epsilon_i$$

Esta última expresión es útil pues, al aparecer P(F=0) explicita que la probabilidad de pertenecer a una categoria se define siempre en relación a otra categoría. Esto será útil para la interpretación del modelo multi-categoría.


In [ ]:
%%R
# Levantamos los datos
datosghp <- read.csv("https://raw.githubusercontent.com/rpasquini/urban-econometrics/master/data/formal%20premia%20data%20extract.csv")

# Mostramos los primeros registros
head(datosghp)


In [ ]:
%%R
logit <- function(formula, data) {
  # Función que realiza la regresión Logit
  # formula: fórmula del modelo en formato R (ej: y ~ x1 + x2)
  # data: dataframe con los datos
  
  # Eliminamos filas con NA
  data <- na.omit(data)
  
  # Ajustamos el modelo
  modelo <- glm(formula, data = data, family = binomial(link = "logit"))
  
  # Mostramos el resumen
  print(summary(modelo))
  
  return(modelo)
}

# Ajustamos el modelo
modelo_ajustado <- logit(bformal ~ ingresohogar2 + npersonashogar + dedu_seccomp + dedu_ter_o_mas, 
                        data = datosghp)

# Calculamos los efectos marginales promedio
margins_modelo <- margins(modelo_ajustado)
summary(margins_modelo)


¿Cuál es la probabilidad de que una familia de que una familia con cuatro integrantes, ingreso del hogar de $14000, y jefe con secundario incompleto viva en la formalidad?


In [ ]:
%%R
# Creamos un nuevo dataframe con los valores para predecir
nuevos_datos <- data.frame(
  ingresohogar2 = 14000,
  npersonashogar = 4,
  dedu_seccomp = 0,
  dedu_ter_o_mas = 0
)

# Calculamos la probabilidad
prob <- predict(modelo_ajustado, newdata = nuevos_datos, type = "response")
print(paste("Probabilidad de vivir en la formalidad:", round(prob, 4)))


# Versión Probit
El modelo Probit representa una alternativa al modelo Logit. La función que realiza la transformación en este caso es la [función acumulativa](https://en.wikipedia.org/wiki/Cumulative_distribution_function) de la [Distribución Normal Estandar](https://en.wikipedia.org/wiki/Normal_distribution), que es típicamente denotada $\Phi$. Es decir:

$$P(F=1)=\Phi(\beta_0+\beta_1IngresoHogar_i+\beta_2TamañoHogar_i+\beta_3DSecundarioCompleto_i+\beta_4DTerciarioCompleto_i+\epsilon_i)$$


In [ ]:
%%R
probit <- function(formula, data) {
  # Función que realiza la regresión Probit
  # formula: fórmula del modelo en formato R (ej: y ~ x1 + x2)
  # data: dataframe con los datos
  
  # Eliminamos filas con NA
  data <- na.omit(data)
  
  # Ajustamos el modelo
  modelo <- glm(formula, data = data, family = binomial(link = "probit"))
  
  # Mostramos el resumen
  print(summary(modelo))
  
  return(modelo)
}

# Ajustamos el modelo probit
modelo_probit <- probit(bformal ~ ingresohogar2 + npersonashogar + dedu_seccomp + dedu_ter_o_mas, 
                       data = datosghp)

# Calculamos los efectos marginales promedio
margins_probit <- margins(modelo_probit)
summary(margins_probit)


# Estimando un Modelo de Localización Residencial en la Ciudad. Aplicación del modelo de múltiples categorías.

El modelo a estimar será un Multinomial Logit. Este modelo se puede pensar como una generalización del modelo Logit para una cantidad de K categorías. Podemos pensar a este modelo como la modelización de K-1 ecuaciones tomando como base a una categoría de referencia (la K-esima).  

Al igual que el Logit, este modelo tiene la propiedad de que el logaritmo del ratio de las chances entre las categorías se modela de manera lineal. En este caso, el logaritmo del ratio de las chances de una categroria k-esima en relación a la categoría K (la última), se puede escribir como:

$$ \ln(\frac{\Pr(Y=k)}{\Pr(Y=K)})=\beta_{0,k}+\beta_{1,k}X_i+\epsilon_i$$

El modelo tiene similares implicancias al Logit de dos categorías: Los efectos del modelo en las probabilidades serán no-lineales (i.e., dependerán de los valores de las variables explicativas) y será necesario estimar los *efectos marginales*.


In [ ]:
%%R
# Descargamos y leemos el shapefile de comunas
temp <- tempfile()
download.file("https://github.com/rpasquini/econometrics_and_causality/blob/master/comunas.zip?raw=true",
              temp)
unzip(temp)
comunas <- st_read("comunas.shp")

# Visualizamos el mapa de comunas
ggplot() +
  geom_sf(data = comunas, aes(fill = factor(COMUNAS))) +
  scale_fill_viridis_d() +
  theme_minimal() +
  labs(title = "Comunas de Buenos Aires",
       fill = "Comuna")


In [ ]:
%%R
# Cargamos los datos de la EAH
dataeah <- read.csv("https://github.com/rpasquini/econometrics_and_causality/blob/master/eah2016.csv?raw=true")

# Filtramos para quedarnos solo con los jefes de hogar
dataeah <- dataeah[dataeah$parentes_2 == 1,]

# Mostramos las primeras filas
head(dataeah)


In [ ]:
%%R
# Función para ajustar el modelo multinomial logit
mnlogit <- function(formula, data) {
  # Eliminamos filas con NA
  data <- na.omit(data)
  
  # Ajustamos el modelo
  modelo <- multinom(formula, data = data)
  
  # Mostramos el resumen
  print(summary(modelo))
  
  return(modelo)
}

# Ajustamos el modelo multinomial
modelo_mn <- mnlogit(comuna ~ ingtot_2 + edad, data = dataeah)

# Calculamos probabilidades predichas para un caso específico
nuevos_datos <- data.frame(
  ingtot_2 = 20000,
  edad = 60
)

# Predicción
predicciones <- predict(modelo_mn, newdata = nuevos_datos, type = "probs")
print("Probabilidades predichas por comuna:")
print(predicciones)
